This notebook takes the well data stored in a local database and renders out a binary file with the well location data in EarthTime format.

In [1]:
import os, array, csv, json, math, random
from datetime import datetime

import db_settings
import psycopg2

def LonLatToPixelXY(lonlat):
    (lon, lat) = lonlat
    x = (lon + 180.0) * 256.0 / 360.0
    y = 128.0 - math.log(math.tan((lat + 90.0) * math.pi / 360.0)) * 128.0 / math.pi
    return [x, y]

## Traditional - Lat / Lon / Time

In [2]:
conn = psycopg2.connect(database=db_settings.DB, user=db_settings.USER, password=db_settings.PASSWD, host=db_settings.HOST)
with conn:
    with conn.cursor() as cur:
        query = 'SELECT date, ST_X(location::geometry) AS lon, ST_Y(location::geometry) AS lat FROM wells '
        query += "WHERE type IN ('OIL', 'GAS', 'OILANDGAS')"

        try:
            cur.execute(query)
        except psycopg2.Error as e:
            print (query)
            print (e.pgerror)
        wells = cur.fetchall()
conn.close()

In [3]:
data = []
base_date = datetime(1970, 1, 1)
for well in wells:
    date, lon, lat = well
    x, y = LonLatToPixelXY([lon,lat])
    if date:
        date = datetime.combine(date, datetime.min.time())
        epochtime = (date - base_date).total_seconds()
    else:
        epochtime = 0
    data += [x,y,epochtime]
array.array('f', data).tofile(open('data/data.bin', 'wb'))

# Experimental - Lat / Lon / Time / Active

In [5]:
# Extract well data from database and write to .bin file

import db_settings
import psycopg2, psycopg2.extras, ast, math, array
from datetime import datetime
result_file = 'data/decay.bin'

query = """
SELECT DISTINCT ON(api) api, ST_AsGeoJSON(location) AS location, 
date,
date_part('year', date) AS year,
date_part('month', date) AS month,
date_part('day', date) AS day,
CASE WHEN status = 'ACTIVE' THEN 'True' ELSE 'False' END AS active
FROM wells
WHERE type IN ('OIL', 'OILANDGAS', 'GAS')
AND date IS NOT NULL
ORDER BY api, date, capture_time DESC;
"""
data = []
conn = psycopg2.connect(database=db_settings.DB, user=db_settings.USER, password=db_settings.PASSWD, host=db_settings.HOST)
with conn:
    with conn.cursor(name='wells_cur', cursor_factory=psycopg2.extras.DictCursor) as cur:
        cur.execute(query)
        for record in cur:
            if record['date']:
                date = datetime(int(record['year']), int(record['month']), int(record['day']))
            epochtime = (date - datetime(1970, 1, 1)).total_seconds()
            value = 1 if ast.literal_eval(record['active']) else 0            
            point = ast.literal_eval(record['location'])
            x = (point['coordinates'][0] + 180.0) * 256.0 / 360.0
            y = 128.0 - math.log(math.tan((point['coordinates'][1] + 90.0) * math.pi / 360.0)) * 128.0 / math.pi
            data += [x, y, epochtime, value];

array.array('f', data).tofile(open(result_file, 'wb'))
print ('wrote', str(len(data)/4), 'records to', result_file)

wrote 1374085.0 records to data/decay.bin
